In [ ]:
!pip install google-cloud-storage
from google.cloud import storage
!gcloud auth application-default login --quiet
!gcloud config set project medfuse-456700 --quiet


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=SGjRWphpYYFVjooe8xiTkYBERm72Ng&prompt=consent&token_usage=remote&access_type=offline&code_challenge=lJi9jXaknwuPQZv02-5k-c5Yx_4v0fqW4tFMqSQ6kR4&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0Ab_5qlllzKPMTUrwnapYDoZ4r-XXMwl_27_mi-5LXV3rGkgkxBqU1FGMo3KhFH0HIjzc2w

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
gsutil ls  "gs://medfuse-mimic-cxr/resized images/**/p17*.jpg" | wc -l

SyntaxError: invalid syntax (<ipython-input-4-f1285691741d>, line 1)

In [ ]:
#!gsutil -u medfuse-456700 -m cp -r \
    #gs://mimic-cxr-jpg-2.1.0.physionet.org/* \
    #gs://medfuse-mimic-cxr/

ServiceException: 401 Requester pays bucket access requires authentication.
CommandException: 1 file/object could not be transferred.


Importing the libraries

In [ ]:
import os
import subprocess
env = os.environ.copy()

def run(cmd):
    """Run a shell command via subprocess, echoing output."""
    print(f"> {' '.join(cmd)}")
    subprocess.run(cmd, check=True,env=env)


# 1. Authenticate ADC
!{("gcloud "+ "auth" " application-default"+" login")}

# 2. Log in your user account
!{["gcloud", "auth", "login"]}



print("✅ All done!")


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=xY3xJSCW9vhzd9cqIp72yEZdMTwRmh&prompt=consent&token_usage=remote&access_type=offline&code_challenge=ItvF7w8TCwnFgOy6VzADzbcrb4DMwcNYd7Rsl4jMVtA&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 

In [ ]:
# Authenticate ADC
!gcloud auth application-default login --quiet

! gcloud auth login

# Sync with billing-project flag
!gcloud storage --billing-project=medfuse-456700 rsync --recursive \
    gs://mimic-cxr-jpg-2.1.0.physionet.org/ gs://medfuse-mimic-cxr/


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=V6tf0ZcSP97PapBI6fQyW8SSHl2Jbt&prompt=consent&token_usage=remote&access_type=offline&code_challenge=HqbRt27zeO68NXNWuH25UFAk_dvXRpUpMURu2HJZOL0&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0Ab_5qlnVc_qprWBq8ED_MuIARWrx7t1vSvXqKJDcFIp-sfdJg48fmwYduhnjGYgklvCKwA

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Q

In [ ]:
!pip install gcsfs
import os
import gcsfs
from tqdm import tqdm
from PIL import Image
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor, as_completed

Defining the variables for the downloading the data

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from IPython.display import clear_output
# --- CONFIGURATION ---
TARGET_WIDTH = 512
MAX_WORKERS = 8
SOURCE_BUCKET_PREFIX = 'medfuse-mimic-cxr/files'
DEST_BUCKET_PREFIX = 'medfuse-mimic-cxr/resized images'  # 🔁 Update your bucket name here

# Set up filesystem
fs = gcsfs.GCSFileSystem(token='google_default')  # Credentialed user
all_jpg_files=[]


In [ ]:
# --- STEP 1: List all .jpg files in source bucket ---
all_jpg_files = [f for f in fs.find(SOURCE_BUCKET_PREFIX) if f.endswith('.jpg')]

# --- STEP 2: Create flat filenames ---
rel_paths = [os.path.relpath(f, SOURCE_BUCKET_PREFIX) for f in all_jpg_files]
flat_filenames = [rp.replace('/', '_') for rp in rel_paths]
full_path_map = dict(zip(flat_filenames, all_jpg_files))  # flattened name → GCS full path

Download the files

In [ ]:
#Create flat filenames ---
rel_paths = [os.path.relpath(f, SOURCE_BUCKET_PREFIX) for f in all_jpg_files]
flat_filenames = [rp.replace('/', '_') for rp in rel_paths]
full_path_map = dict(zip(flat_filenames, all_jpg_files))  # flattened name → GCS full path

# --- STEP 3: Find missing files in DEST_BUCKET ---
try:
    existing_files = set([os.path.basename(f) for f in fs.find(DEST_BUCKET_PREFIX)])
except Exception:
    existing_files = set()

missing_files = sorted(set(flat_filenames) - existing_files)

print(f" Total GCS files: {len(flat_filenames)}")
print(f"Already resized in dest bucket: {len(existing_files)}")
print(f"Missing to process: {len(missing_files)}")

🟡 Total GCS files: 377110
✅ Already resized in dest bucket: 377111
🆕 Missing to process: 0


In [ ]:
def process_and_save_to_gcs(flat_filename):
    src_full_path = full_path_map[flat_filename]
    dest_full_path = f"{DEST_BUCKET_PREFIX}/{flat_filename}"

    try:
        with fs.open(src_full_path, 'rb') as fsrc:
            img = Image.open(fsrc).convert('RGB')
            ratio = TARGET_WIDTH / img.width
            new_h = int(img.height * ratio)
            resized = img.resize((TARGET_WIDTH, new_h), Image.LANCZOS)

        # Save to destination GCS bucket
        buffer = BytesIO()
        resized.save(buffer, format='JPEG')
        buffer.seek(0)

        with fs.open(dest_full_path, 'wb') as fdst:
            fdst.write(buffer.read())

        return f"Saved: {dest_full_path}"
    except Exception as e:
        return f"Failed: {src_full_path} – {str(e)}"

# --- STEP 5: Parallel execution ---
print(f" Resizing and uploading {len(missing_files)} images with {MAX_WORKERS} workers...")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(process_and_save_to_gcs, fn) for fn in missing_files]
    for future in tqdm(as_completed(futures), total=len(futures)):
        print(future.result())
        clear_output(wait=True)


print(f" Done. Resized images uploaded to: gs://{DEST_BUCKET_PREFIX}/")

100%|██████████| 8610/8610 [07:30<00:00, 19.11it/s]


🎉 Done. Resized images uploaded to: gs://medfuse-mimic-cxr/resized images/


In [ ]:
# 1️⃣ Authenticate to Google Cloud
from google.colab import auth
auth.authenticate_user()


# 3️⃣ Define paths
GCS_BUCKET_PATH = 'gs://medfuse-mimic-cxr/resized images'            # ← adjust to your bucket/folder
GDRIVE_DEST_DIR  = '/content/drive/My Drive/CS 598 DFLH - Final Project - Medfuse/'  # ← adjust to your Drive folder

# 4️⃣ Ensure the destination exists
import os
os.makedirs(GDRIVE_DEST_DIR, exist_ok=True)

# 5️⃣ Copy recursively using gsutil
#    -m: multi-threaded
#    -r: recursive
#    trailing /*: copy contents, not the folder container itself
!gsutil -m cp -r "{GCS_BUCKET_PATH}/*" "{GDRIVE_DEST_DIR}/"

Streaming output truncated to the last 5000 lines.
Copying gs://medfuse-mimic-cxr/resized images/p18_p18699864_s50003755_371607a8-16c45526-b8e74381-02d93e76-50f64289.jpg...
Copying gs://medfuse-mimic-cxr/resized images/p18_p18699864_s50440582_f9713418-f00ef7e0-1290f359-03414b25-4727c1b6.jpg...
Copying gs://medfuse-mimic-cxr/resized images/p18_p18699864_s51656628_1bf1daf8-ac46d614-ba01e478-c52e226e-288a860e.jpg...
Copying gs://medfuse-mimic-cxr/resized images/p18_p18699864_s51656628_6b3ac860-fba86b86-4cd8cddf-c788c883-f22bf3df.jpg...
Copying gs://medfuse-mimic-cxr/resized images/p18_p18699864_s51915401_0dfee634-15a98608-ac3b46c5-31388788-bebe1271.jpg...
Copying gs://medfuse-mimic-cxr/resized images/p18_p18699864_s51915401_dcefa9c4-d7b3ae1b-b0df4ee5-d88365bf-122e59de.jpg...
Copying gs://medfuse-mimic-cxr/resized images/p18_p18699864_s51915401_e647b050-cea45ebe-d3b3c10f-9a457872-7f22c019.jpg...
Copying gs://medfuse-mimic-cxr/resized images/p18_p18699864_s52598516_70545b3d-8a18050a-bc68b56

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls -ltr drive/MyDrive


total 3705827
-rw------- 1 root root     590373 Sep 21  2009 'RMS Documents.zip'
-rw------- 1 root root        176 Jan 21  2010 'Implementation of Computing Intensive....gdoc'
-rw------- 1 root root        176 Apr 22  2010 'Alumini List.gsheet'
-rw------- 1 root root        176 Sep 12  2010  GunmeetSinghResume091009-v1.3.doc.gdoc
-rw------- 1 root root        176 Sep 12  2010  GunmeetSinghResume091009-v1.4.doc.gdoc
-rw------- 1 root root        176 Feb  3  2012 'Untitled document (2).gdoc'
-rw------- 1 root root        176 Nov  5  2012 'E Shop Task List.gsheet'
-rw------- 1 root root        176 Mar 18  2013 'Untitled form.gform'
-rw------- 1 root root        176 Dec 30  2013 'Manali to Pangong Tso Planning trip.gsheet'
-rw------- 1 root root   85576774 Feb 16  2014  DSC_0444.tif
-rw------- 1 root root        176 Apr 13  2014 'Man of Steel Movie Premiere.gsheet'
-rw------- 1 root root        176 Oct 15  2014 'Untitled form (Responses).gsheet'
-rw------- 1 root root        176 Dec 20  20